In [ ]:
# loading the data

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt

In [2]:
# using glob to extract all .edf files from the folder

all_file_path = glob('data/*.edf')
print(len(all_file_path))

72


In [3]:
all_file_path[:5]

['data\\Subject00_1.edf',
 'data\\Subject00_2.edf',
 'data\\Subject01_1.edf',
 'data\\Subject01_2.edf',
 'data\\Subject02_1.edf']

In [4]:
# from data set we already know there are total 36 persons 
# and each one has two .edf files subjectid_01 for data before arithmetic task
# and subjectid_02 for data during arithmetic task
# so here i assigned all the files which has 1 after '_' to before_arithmetic_task
# similarly all the files that has 2 after '_' to during arithmetic task

before_arithmetic_task = [i for i in all_file_path if '1' in i.split('_')[1]]
during_arithmetic_task = [i for i in all_file_path if '2' in i.split('_')[1]]

print(len(before_arithmetic_task), len(during_arithmetic_task))

36 36


In [5]:
# function to read the .edf file data using mne package
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload = True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 45)
    epochs = mne.make_fixed_length_epochs(data, duration = 5, overlap = 1)
    array = epochs.get_data()
    return array

In [6]:
# i just passed a edf file to read_data function to check weather the function is working or not
# we can see what information is extracted from the .edf file

sample_data = read_data(during_arithmetic_task[0])

Extracting EDF parameters from C:\Users\prave\Desktop\eeg-during-mental-arithmetic-tasks\data\Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 3301 samples (6.602 s)

Not setting metadata
15 matching events found
No baseline correction appl

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.0s finished


In [7]:
# shape of the data that we extracted from .edf file
sample_data.shape # no of epochs, channels, length of signal

(15, 21, 2500)

In [8]:
# the following two lines are needed in order to execute the magic word 'capture'
# it function is to capture the cell output and hides it 
# used to hide the unnecessary info from printing 
# it has no connection to this project

import nest_asyncio
nest_asyncio.apply()

In [9]:
%%capture

# extract all 36 before_arithmetic_task data into before_epochs_array
before_epochs_array = [read_data(i) for i in before_arithmetic_task]

# extract all 36 during_arithmetic_task data into during_epochs_array
during_epochs_array = [read_data(i) for i in during_arithmetic_task]

In [10]:
before_epochs_array[0].shape, during_epochs_array[0].shape # no of epochs, channels, length of signal

((45, 21, 2500), (15, 21, 2500))

In [11]:
# each person data be in the following format
# person 1
#    epoch 1
#    epoch 2
#    epoch 3
#    ....
# person 2
#    epoch 1
#    epoch 2
#    epoch 3
#    ....

# so we need to create label for each epoch under each person not just one label for one person
# the following two lines of code are to create labels for each epoch under each person/each .edf file
# 0 for before arithmetic task and 1 for during arithmetic task data


before_epochs_labels = [len(i)*[0] for i in before_epochs_array]
during_epochs_labels = [len(i)*[1] for i in during_epochs_array]
len(before_epochs_labels), len(during_epochs_labels)

(36, 36)

In [12]:
# data_list is combination of before arithmetic data and during arithmetic data
data_list = before_epochs_array + during_epochs_array

# label_list is combination of before arithmetic labels and during arithmetic labels
label_list = before_epochs_labels + during_epochs_labels

In [13]:
# split the data based on subjects
# not on the basis of epochs or trails
# assign a group to each subject(edf file) and split based on groups
# because we know the data is in the follwing format
# person 1
#    epoch 1
#    epoch 2
#    epoch 3
#    ....
# person 2
#    epoch 1
#    epoch 2
#    epoch 3
#    ....
# if we randomly split it based on epochs person1 epoch1 may fall under training data
# at the same time person1 epoch2 may fall under test data
# now the model would easily predict because it trained on person1 and leads to overfitting

# in order to overcome that we assign a group value for each epoch
# like person1 group value for each epoch is 1 and person2 group value for each epoch is 2 so on
# and then split the data based on groups 
# now all the epochs on person1 may fall either on train or test not on both
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

72

In [14]:
# convert lists into numpy arrays
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)
print(data_array.shape, label_array.shape, group_array.shape)

(2132, 21, 2500) (2132,) (2132,)


In [15]:
# 2132 no of segments, 21 no of channels, 2500 is the length
# cnn expects the channel at the end so we need to change it to (2132, 2500, 21)
data_array=np.moveaxis(data_array,1,2)
data_array.shape

(2132, 2500, 21)

In [21]:
# main goal for this project is to classify weather given .edf file data is belongs to a person
# who is in rest state i.e before arithmetic task 
# or task state i.e during arithmetic task
# due to the values are too small standard scalar is applied in order to scale the data

# custom cnn model using tensorflow, keras
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(2500,21))) # input shape ((2132, 2500, 21))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))
    model.add(Dropout(0.5))
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))
    model.add(Dropout(0.5))
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(1,activation='sigmoid')) # sigmoid is more suitable for binary classification
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2498, 5)           320       
                                                                 
 batch_normalization (Batch  (None, 2498, 5)           20        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 2498, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 1249, 5)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 1247, 5)           80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 1247, 5)           0

In [22]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [25]:
accuracy=[] # to store accuracy for each fold

for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    
    # from gkf.split we can get train, test and validation data
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    
    # after the shape of train_features will be like (1710, 2500, 21)
    # for scalling we can only apply standard scaler on 2d data
    # so in order to apply standard we have to convert train_features into 2d array (1710*2500, 21)
    # then apply scalling
    # after scalling is done again reshape the data into its original form like (1710, 2500, 21)
    
    scaler=StandardScaler()
    
    # reshape train_features into 2d apply scalling then reshape it back into original format
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    
    # reshape val_features into 2d apply scalling then reshape it back into original format
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)

    # train the model
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=128,validation_data=(val_features,val_labels))
    
    # evaluate and append the accuracy value of this batch
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
14/14 [==============================] - 8s 148ms/step - loss: 0.6006 - accuracy: 0.7368 - val_loss: 0.5349 - val_accuracy: 0.7867
Epoch 2/50
14/14 [==============================] - 2s 122ms/step - loss: 0.5680 - accuracy: 0.7368 - val_loss: 0.5054 - val_accuracy: 0.7867
Epoch 3/50
14/14 [==============================] - 2s 115ms/step - loss: 0.5637 - accuracy: 0.7368 - val_loss: 0.5033 - val_accuracy: 0.7867
Epoch 4/50
14/14 [==============================] - 2s 130ms/step - loss: 0.5628 - accuracy: 0.7368 - val_loss: 0.5077 - val_accuracy: 0.7867
Epoch 5/50
14/14 [==============================] - 3s 197ms/step - loss: 0.5625 - accuracy: 0.7368 - val_loss: 0.5040 - val_accuracy: 0.7867
Epoch 6/50
14/14 [==============================] - 3s 200ms/step - loss: 0.5613 - accuracy: 0.7368 - val_loss: 0.5033 - val_accuracy: 0.7867
Epoch 7/50
14/14 [==============================] - 3s 197ms/step - loss: 0.5597 - accuracy: 0.7368 - val_loss: 0.5071 - val_accuracy: 0.7867
Epoch 

Epoch 8/50
14/14 [==============================] - 3s 234ms/step - loss: 0.5303 - accuracy: 0.7525 - val_loss: 0.6021 - val_accuracy: 0.7241
Epoch 9/50
14/14 [==============================] - 3s 249ms/step - loss: 0.5269 - accuracy: 0.7525 - val_loss: 0.6098 - val_accuracy: 0.7241
Epoch 10/50
14/14 [==============================] - 3s 232ms/step - loss: 0.5256 - accuracy: 0.7525 - val_loss: 0.6079 - val_accuracy: 0.7241
Epoch 11/50
14/14 [==============================] - 3s 211ms/step - loss: 0.5210 - accuracy: 0.7525 - val_loss: 0.6135 - val_accuracy: 0.7241
Epoch 12/50
14/14 [==============================] - 3s 213ms/step - loss: 0.5161 - accuracy: 0.7525 - val_loss: 0.6114 - val_accuracy: 0.7241
Epoch 13/50
14/14 [==============================] - 3s 222ms/step - loss: 0.5126 - accuracy: 0.7525 - val_loss: 0.6144 - val_accuracy: 0.7241
Epoch 14/50
14/14 [==============================] - 3s 216ms/step - loss: 0.5089 - accuracy: 0.7525 - val_loss: 0.6218 - val_accuracy: 0.7241
E

Epoch 15/50
14/14 [==============================] - 3s 201ms/step - loss: 0.5410 - accuracy: 0.7525 - val_loss: 0.5644 - val_accuracy: 0.7241
Epoch 16/50
14/14 [==============================] - 3s 200ms/step - loss: 0.5415 - accuracy: 0.7525 - val_loss: 0.5648 - val_accuracy: 0.7241
Epoch 17/50
14/14 [==============================] - 3s 213ms/step - loss: 0.5397 - accuracy: 0.7525 - val_loss: 0.5649 - val_accuracy: 0.7241
Epoch 18/50
14/14 [==============================] - 3s 183ms/step - loss: 0.5400 - accuracy: 0.7525 - val_loss: 0.5637 - val_accuracy: 0.7241
Epoch 19/50
14/14 [==============================] - 2s 118ms/step - loss: 0.5390 - accuracy: 0.7525 - val_loss: 0.5642 - val_accuracy: 0.7241
Epoch 20/50
14/14 [==============================] - 2s 115ms/step - loss: 0.5392 - accuracy: 0.7525 - val_loss: 0.5656 - val_accuracy: 0.7241
Epoch 21/50
14/14 [==============================] - 2s 133ms/step - loss: 0.5380 - accuracy: 0.7525 - val_loss: 0.5644 - val_accuracy: 0.7241

Epoch 22/50
14/14 [==============================] - 3s 203ms/step - loss: 0.4345 - accuracy: 0.7985 - val_loss: 0.6830 - val_accuracy: 0.6976
Epoch 23/50
14/14 [==============================] - 3s 204ms/step - loss: 0.4204 - accuracy: 0.8113 - val_loss: 0.6926 - val_accuracy: 0.7071
Epoch 24/50
14/14 [==============================] - 3s 206ms/step - loss: 0.4203 - accuracy: 0.8026 - val_loss: 0.6936 - val_accuracy: 0.6810
Epoch 25/50
14/14 [==============================] - 3s 206ms/step - loss: 0.4201 - accuracy: 0.8102 - val_loss: 0.6886 - val_accuracy: 0.7000
Epoch 26/50
14/14 [==============================] - 3s 214ms/step - loss: 0.4076 - accuracy: 0.8143 - val_loss: 0.6879 - val_accuracy: 0.6833
Epoch 27/50
14/14 [==============================] - 3s 217ms/step - loss: 0.4002 - accuracy: 0.8213 - val_loss: 0.6950 - val_accuracy: 0.6571
Epoch 28/50
14/14 [==============================] - 3s 222ms/step - loss: 0.4026 - accuracy: 0.8236 - val_loss: 0.6929 - val_accuracy: 0.6786

Epoch 29/50
14/14 [==============================] - 3s 200ms/step - loss: 0.5413 - accuracy: 0.7459 - val_loss: 0.5227 - val_accuracy: 0.7500
Epoch 30/50
14/14 [==============================] - 3s 208ms/step - loss: 0.5404 - accuracy: 0.7459 - val_loss: 0.5220 - val_accuracy: 0.7500
Epoch 31/50
14/14 [==============================] - 3s 206ms/step - loss: 0.5402 - accuracy: 0.7459 - val_loss: 0.5220 - val_accuracy: 0.7500
Epoch 32/50
14/14 [==============================] - 3s 206ms/step - loss: 0.5389 - accuracy: 0.7459 - val_loss: 0.5218 - val_accuracy: 0.7500
Epoch 33/50
14/14 [==============================] - 3s 201ms/step - loss: 0.5381 - accuracy: 0.7459 - val_loss: 0.5207 - val_accuracy: 0.7500
Epoch 34/50
14/14 [==============================] - 3s 210ms/step - loss: 0.5407 - accuracy: 0.7459 - val_loss: 0.5195 - val_accuracy: 0.7500
Epoch 35/50
14/14 [==============================] - 3s 199ms/step - loss: 0.5374 - accuracy: 0.7459 - val_loss: 0.5194 - val_accuracy: 0.7500

In [26]:
np.mean(accuracy)

0.7080005288124085

In [ ]:
# 70.8% accuracy is achieved for cnn model